In [1]:
import pandas as pd
import sys
from pwn import *

In [2]:
context.arch = "amd64"
context.log_level = 'CRITICAL'

In [3]:
def rm_slash(op):
    op = op.strip()
    if '/' in op:
        return op.split('/')[0].strip()
    else:
        return op
    
def bytesToHexStr(hex_asm):
    return '\\x'+'\\x'.join(['{:02x}'.format(byte) for byte in hex_asm])

In [4]:
df = pd.read_excel(
        "instruction_tables.ods", 
        sheet_name="Skylake", 
        header=15, 
        usecols=[0, 1, 2, 3], 
        names=['Instruction', 'Operands', 'Fused UOPS', 'Unfused UOPS'])

In [5]:
for i in range(len(df['Unfused UOPS'])):
    if type(df['Unfused UOPS'][i]) == type('string'):
        df['Unfused UOPS'][i] = 999

In [6]:
new_df = df.sort_values('Unfused UOPS')

In [7]:
op1_switch = {
    'i'   : "0x41",
    'r'   : "rbx",
    'r8'  : "bl",
    'r8h' : "bh",
    'r8l' : "bl",
    'r16' : "bx",
    'r32' : "ebx",
    'r64' : "rbx",
    'm'   : "[rsp]",
    'm8'  : "[rsp]",
    'm16' : "[rsp]",
    'm32' : "[rsp]",
    'm64' : "[rsp]",
    'm128': "[rsp]",
    'm256': "[rsp]",
    'mm'  : "xmm0",
    'x'   : "xmm0",
    'y'   : "xmm0",
    'v'   : "xmm0",
    'stack pointer' : "rsp",
    'short/near'    : "rsp",
    'short'         : "rsp",
    'near': "rsp",
    'AX'  : "rax"
}

In [8]:
op2_switch = {
    'i'   : "0x41",
    'r'   : "rcx",
    'r8'  : "cl",
    'r8h' : "ch",
    'r8l' : "cl",
    'r16' : "cx",
    'r32' : "ecx",
    'r64' : "rcx",
    'm'   : "[rsp]",
    'm8'  : "[rsp]",
    'm16' : "[rsp]",
    'm32' : "[rsp]",
    'm64' : "[rsp]",
    'm128': "[rsp]",
    'm256': "[rsp]",
    'mm'  : "xmm1",
    'x'   : "xmm1",
    'y'   : "xmm1",
    'v'   : "xmm1",
    '1'   : "0x1",
    'cl'  : "cl"
}

In [9]:
op3_switch = {
    'i'   : "0x41",
    'r'   : "rdx",
    'r8'  : "dl",
    'r8h' : "dh",
    'r8l' : "dl",
    'r16' : "dx",
    'r32' : "edx",
    'r64' : "rdx",
    'm'   : "[rsp]",
    'm8'  : "[rsp]",
    'm16' : "[rsp]",
    'm32' : "[rsp]",
    'm64' : "[rsp]",
    'm128': "[rsp]",
    'm256': "[rsp]",
    'mm'  : "xmm2",
    'x'   : "xmm2",
    'y'   : "xmm2",
    'v'   : "xmm2",
    'xmm0': "xmm0",
    'cl'  : "cl"
}

In [13]:
new_df.reset_index(drop=True, inplace=True)

for index in new_df.index:
    value = new_df.loc[index, 'Instruction']
    
    if type(value) is type('str'):
        if not value.isupper():
            new_df.drop(index, inplace=True)
            print(value)
    else:
        new_df.drop(index, inplace=True)
        print(value)
        
new_df.reset_index(drop=True, inplace=True)

In [14]:
latest_df = new_df
idx_list = list(latest_df.index)
t = 0
idx_len = len(idx_list)

while t < idx_len:
    index = t
    value = latest_df.loc[index, 'Instruction']
    instr_row = {'Operands': latest_df.loc[index, 'Operands'], 'Unfused UOPS': latest_df.loc[index, 'Unfused UOPS']}
    
    if type(value) is type('str'):
        sep1 = 'B/W/D/Q'
        sep2 = 'W/D/Q'
        sep3 = 'B/W/D'
        
        if ' ' in value and 'REP' not in value and 'LOCK' not in value:
            latest_df.drop(index, inplace=True)
            t -= 1
            print('\n')
            print(value)
            instr_arr = value.split(' ')
            
            
            if '/' in instr_arr[0]:
                sls_split = instr_arr[0].split('/')
                instr_arr[0] = sls_split[0]
                instr_arr.append(sls_split[1])
            
            num_rm = len(instr_arr[1])
            str_rm = instr_arr[0][:-num_rm]
            
            print(instr_arr)
            for instr in instr_arr:
                if str_rm not in instr and len(instr) == num_rm:
                    instr = str_rm + instr
                if len(instr) > 2:
                    instr_row['Instruction'] = instr
                    latest_df = latest_df.append(instr_row, ignore_index=True)
                    print(instr)
        
        
        elif sep1 in value:
            latest_df.drop(index, inplace=True)
            t -= 1
            instr_arr = [value.strip(sep1)]
            print('\n')
            print(value)
            print(instr_arr)
            
            for instr in instr_arr:
                for j in sep1.split('/'):
                    instr_row['Instruction'] = instr+j
                    latest_df = latest_df.append(instr_row, ignore_index=True)
                    print(instr+j)
            
            
        
        
        elif sep2 in value:
            latest_df.drop(index, inplace=True)
            t -= 1
            instr_arr = value.split(sep2)
            print('\n')
            print(value)
            print(instr_arr)
            
            for instr in instr_arr:
                for j in sep2.split('/'):
                    if len(instr) > 2:
                        instr_row['Instruction'] = instr+j
                        latest_df = latest_df.append(instr_row, ignore_index=True)
                        print(instr+j)
        
        
        elif sep3 in value:
            latest_df.drop(index, inplace=True)
            t -= 1
            instr_arr = value.split(sep3)
            print('\n')
            print(value)
            print(instr_arr)
            
            for instr in instr_arr:
                for j in sep3.split('/'):
                    if len(instr) > 2: 
                        instr_row['Instruction'] = instr+j
                        latest_df = latest_df.append(instr_row, ignore_index=True)
                        print(instr+j) 
            
            
        elif '/' in value:
            latest_df.drop(index, inplace=True)
            t -= 1
            print('\n')
            print(value)
            instr_arr = value.split('/')
            
            num_rm = len(instr_arr[1])
            str_rm = instr_arr[0][:-num_rm]
            
#             print('\n')
            print(instr_arr)
            
            for instr in instr_arr:
                if str_rm not in instr and len(instr) == num_rm:
                    if len(instr) < 3: instr = str_rm + instr
                if len(instr) > 2:
                    instr_row['Instruction'] = instr
                    latest_df = latest_df.append(instr_row, ignore_index=True)
                    print(instr)
                    
    t += 1



PALIGNR 
['PALIGNR', '']
PALIGNR


PSHUFW 
['PSHUFW', '']
PSHUFW


VPMOVSX/ZX BW BD BQ DW DQ
['VPMOVSX', 'BW', 'BD', 'BQ', 'DW', 'DQ', 'ZX']
VPMOVSX
VPMOVBW
VPMOVBD
VPMOVBQ
VPMOVDW
VPMOVDQ
VPMOVZX


VPBROADCASTB/W/D/Q
['VPBROADCAST']
VPBROADCASTB
VPBROADCASTW
VPBROADCASTD
VPBROADCASTQ


VPBROADCASTD/Q
['VPBROADCASTD', 'Q']
VPBROADCASTD
VPBROADCASTQ


VPBROADCASTD/Q
['VPBROADCASTD', 'Q']
VPBROADCASTD
VPBROADCASTQ


PMULL/HWPMULHUW
['PMULL', 'HWPMULHUW']
PMULL
HWPMULHUW


PMULL/HWPMULHUW
['PMULL', 'HWPMULHUW']
PMULL
HWPMULHUW


PCMPEQB/W/DPCMPGTB/W/D
['PCMPEQ', 'PCMPGT', '']
PCMPEQB
PCMPEQW
PCMPEQD
PCMPGTB
PCMPGTW
PCMPGTD


PCMPEQB/W/DPCMPGTB/W/D
['PCMPEQ', 'PCMPGT', '']
PCMPEQB
PCMPEQW
PCMPEQD
PCMPGTB
PCMPGTW


/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame

PCMPGTD


PADD/SUB(S,US) B/W/D/Q
['PADD', 'B/W/D/Q', 'SUB(S,US)']
PADD
B/W/D/Q
SUB(S,US)


PMOVSX/ZX BW BD BQ DW DQ
['PMOVSX', 'BW', 'BD', 'BQ', 'DW', 'DQ', 'ZX']
PMOVSX
PMOVBW
PMOVBD
PMOVBQ
PMOVDW
PMOVDQ
PMOVZX


VPBROADCASTB/W/D/Q
['VPBROADCAST']
VPBROADCASTB
VPBROADCASTW
VPBROADCASTD
VPBROADCASTQ


PUNPCKH/LQDQ
['PUNPCKH', 'LQDQ']
PUNPCKH
LQDQ


FINCSTP FDECSTP
['FINCSTP', 'FDECSTP']
FINCSTP
FDECSTP


FCOM(P) FUCOM
['FCOM(P)', 'FUCOM']
FCOM(P)
FCFUCOM


PACKSSWB/DW PACKUSWB
['PACKSSWB', 'PACKUSWB', 'DW']
PACKSSWB
PACKUSWB


VMOVDQA/U
['VMOVDQA', 'U']
VMOVDQA
VMOVDQU


VMOVDQA/U
['VMOVDQA', 'U']
VMOVDQA
VMOVDQU


MOVDQA/U
['MOVDQA', 'U']
MOVDQA
MOVDQU


MOVDQA/U
['MOVDQA', 'U']
MOVDQA
MOVDQU


PUNPCKH/LBW/WD/DQ
['PUNPCKH', 'LBW', 'WD', 'DQ']
PUNPCKH
LBW


PAVGB/W
['PAVGB', 'W']
PAVGB
PAVGW


PAVGB/W
['PAVGB', 'W']
PAVGB
PAVGW


ADDSS/D PS/DSUBSS/D PS/D
['ADDSS', 'PS/DSUBSS/D', 'PS/D', 'D']
ADDSS
PS/DSUBSS/D
PS/D


VCVT(T) PS2DQ
['VCVT(T)', 'PS2DQ']
VCVT(T)


/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame

VCPS2DQ


CVT(T) PS2DQ
['CVT(T)', 'PS2DQ']
CVT(T)
CPS2DQ


UNPCKH/LPS/D
['UNPCKH', 'LPS', 'D']
UNPCKH
LPS


MOVSH/LDUP
['MOVSH', 'LDUP']
MOVSH
LDUP


MOVSH/LDUP
['MOVSH', 'LDUP']
MOVSH
LDUP


ADDSUBPS/D
['ADDSUBPS', 'D']
ADDSUBPS
ADDSUBPD


MULSS/D PS/D
['MULSS', 'PS/D', 'D']
MULSS
MPS/D


AND/ANDN/OR/XORPS/PD
['AND', 'ANDN', 'OR', 'XORPS', 'PD']
AND
ANDN
XORPS


RSQRTSS/PS
['RSQRTSS', 'PS']
RSQRTSS
RSQRTPS


SQRTSS/PS
['SQRTSS', 'PS']
SQRTSS
SQRTPS


(U)COMISS/D
['(U)COMISS', 'D']
(U)COMISS
(U)COMISD


RCPSS/PS
['RCPSS', 'PS']
RCPSS
RCPPS


MAXSS/D PS/DMINSS/D PS/D
['MAXSS', 'PS/DMINSS/D', 'PS/D', 'D']
MAXSS
PS/DMINSS/D
PS/D


VPSLLVD/QVPSRAVDVPSRLVD/Q
['VPSLLVD', 'QVPSRAVDVPSRLVD', 'Q']
VPSLLVD
QVPSRAVDVPSRLVD


PSLLW/D/QPSRLW/D/QPSRAW/D/Q
['PSLL', 'PSRL', 'PSRA', '']
PSLLW
PSLLD
PSLLQ
PSRLW
PSRLD
PSRLQ
PSRAW
PSRAD
PSRAQ


PSLLW/D/QPSRLW/D/QPSRAW/D/Q
['PSLL', 'PSRL', 'PSRA', '']
PSLLW
PSLLD
PSLLQ


...

ROL


SHR SHL SAR
['SHR', 'SHL', 'SAR']
SHR
SHL
SAR


MUL IMUL
['MUL', 'IMUL']
MUL
IMUL


PUSHF(D/Q)
['PUSHF(D', 'Q)']
PUSHF(D
PUSHFQ)


MUL IMUL
['MUL', 'IMUL']
MUL
IMUL


AND OR XOR
['AND', 'OR', 'XOR']
AND
AOR
XOR


SHR SHL SAR
['SHR', 'SHL', 'SAR']
SHR
SHL
SAR


PHADD(S)W/DPHSUB(S)W/D
['PHADD(S)W', 'DPHSUB(S)W', 'D']
PHADD(S)W
DPHSUB(S)W


HADDPS/DHSUBPS/D
['HADDPS', 'DHSUBPS', 'D']
HADDPS
DHSUBPS


BTR BTS BTC
['BTR', 'BTS', 'BTC']
BTR
BTS
BTC


INC DEC NOT
['INC', 'DEC', 'NOT']
INC
DEC
NOT


VMASKMOVPS/D
['VMASKMOVPS', 'D']
VMASKMOVPS
VMASKMOVPD


VMASKMOVPS/D
['VMASKMOVPS', 'D']
VMASKMOVPS
VMASKMOVPD


ADD SUB
['ADD', 'SUB']
ADD
SUB


FIADD FISUB(R)
['FIADD', 'FISUB(R)']
FIADD
FISUB(R)


SHRD SHLD
['SHRD', 'SHLD']
SHRD
SHLD


MUL IMUL
['MUL', 'IMUL']
MUL
IMUL


ROR ROL
['ROR', 'ROL']
ROR
ROL


/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:109: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:109: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The fra



ADC SBB
['ADC', 'SBB']
ADC
SBB


RCR RCL
['RCR', 'RCL']
RCR
RCL


SHR SHL SAR
['SHR', 'SHL', 'SAR']
SHR
SHL
SAR


ROR ROL
['ROR', 'ROL']
ROR
ROL


SHRD SHLD
['SHRD', 'SHLD']
SHRD
SHLD


RCR RCL
['RCR', 'RCL']
RCR
RCL


RCR RCL
['RCR', 'RCL']
RCR
RCL


POPF(D/Q)
['POPF(D', 'Q)']
POPF(D
POPFQ)


BTR BTS BTC
['BTR', 'BTS', 'BTC']
BTR
BTS
BTC


RCR RCL
['RCR', 'RCL']
RCR
RCL


RCR RCL
['RCR', 'RCL']
RCR
RCL


FYL2X 
['FYL2X', '']
FYL2X


B/W/D/Q
['']
B
W
D
Q


PS/DSUBSS/D
['PS', 'DSUBSS', 'D']
DSUBSS


PS/D
['PS', 'D']


/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_df = latest_df.append(instr_row, ignore_index=True)
/tmp/ipykernel_202251/1962900313.py:38: FutureWarning: The frame

KeyError: 473

In [15]:
latest_df = latest_df.sort_values('Unfused UOPS')

In [16]:
latest_df

,Instruction,Operands,Fused UOPS,Unfused UOPS
0,CLC,NaN,1,0
1,FXCH,r,2,0
488,AND,"x,x / v,v,v",NaN,1
487,MPS/D,"x,x / v,v,v",NaN,1
486,MULSS,"x,x / v,v,v",NaN,1
...,...,...,...,...
400,XSAVE,NaN,304,NaN
401,XRSTOR,NaN,257,NaN
402,XRSTOR,NaN,257,NaN
403,XSAVEOPT,m,168,NaN


In [17]:
for num_of_uops in list(set(latest_df['Unfused UOPS'].tolist())):

    instr_list = latest_df.loc[latest_df['Unfused UOPS'] == num_of_uops]['Instruction'].tolist()
    op_list = latest_df.loc[latest_df['Unfused UOPS'] == num_of_uops]['Operands'].tolist()

    print("\n\nFor number of UOPS =", num_of_uops)
    i = 0
    z = 0
    
    while i < len(instr_list):
        print("=> ")
        for (instr, op) in zip(instr_list[i: i+5], op_list[i: i+5]):
            partsOfInstr = []

            if type(op) is type('str'):
                arr_of_operands = op.split(',')
                num_of_operands = len(arr_of_operands)

                op_1 = rm_slash(arr_of_operands[0])
                partsOfInstr.append(op1_switch.get(op_1))

                if num_of_operands > 1:
                    op_2 = rm_slash(arr_of_operands[1])
                    partsOfInstr.append(op2_switch.get(op_2))

                    if num_of_operands > 2:
                        op_3 = rm_slash(arr_of_operands[2])
                        partsOfInstr.append(op3_switch.get(op_3))
            try:
                instruction = instr.lower() + ' ' + ', '.join(partsOfInstr)
                try:
                    hex_asm = asm(instruction)
#                     lines = instruction + " | " + bytesToHexStr(hex_asm) + "\n"
                    lines = instruction + " | "
                    print(lines)
                    
#                     filename = "UOPS-" + str(num_of_uops) + "-instruction-set-" + str(i//5) + ".txt"
                    filename = "NEWUOPS-" + str(num_of_uops) + "-instruction-set" + ".txt"
                    
                    with open(filename, 'a') as f:
                        f.write(lines)
                    with open(filename, 'ab') as f:
                        f.write(hex_asm)
                    with open(filename, 'a') as f:
                        f.write('\n')
                        
                    
                    z += 1
                except Exception as e:
                    print("**", instruction)
#                     print(e)
            except:
                print("**", instr, arr_of_operands)
            
            i += 1
        
    print("\n==== Number of Instructions decoded =", z, "====")



For number of UOPS = 0
=> 
clc  | 
** fxch rbx

==== Number of Instructions decoded = 1 ====


For number of UOPS = 1
=> 
** and xmm0, xmm1, xmm2
** mps/d xmm0, xmm1, xmm2
** mulss xmm0, xmm1, xmm2
** addsubpd xmm0, xmm1, xmm2
** addsubps xmm0, xmm1, xmm2
=> 
** ldup xmm0, xmm1
** q xmm0, xmm1, xmm2
** ldup xmm0, [rsp]
** movsh xmm0, [rsp]
** lps xmm0, xmm1, xmm2
=> 
** unpckh xmm0, xmm1, xmm2
** cps2dq xmm0, xmm1
** cvt(t) xmm0, xmm1
** vcps2dq xmm0, xmm1
** vcvt(t) xmm0, xmm1
=> 
** addss xmm0, xmm1, xmm2
** pavgw xmm0, xmm1, xmm2
** pavgb xmm0, xmm1, xmm2
pavgw xmm0, xmm1 | 
pavgb xmm0, xmm1 | 
=> 
** lbw xmm0, xmm1, xmm2
** punpckh xmm0, xmm1, xmm2
movdqu xmm0, xmm1 | 
movdqa xmm0, xmm1 | 
** andn xmm0, xmm1, xmm2
=> 
** xorps xmm0, xmm1, xmm2
rsqrtss xmm0, xmm1 | 
rsqrtps xmm0, xmm1 | 
psrld xmm0, 0x41 | 
psrlw xmm0, 0x41 | 
=> 
psllq xmm0, 0x41 | 
pslld xmm0, 0x41 | 
psllw xmm0, 0x41 | 
** psraq xmm0, 0x41, xmm2
** psrad xmm0, 0x41, xmm2
=> 
** psraw xmm0, 0x41, xmm2
** psrlq x

** movzx bx, [rsp]
vmaskmovpd xmm0, xmm1, [rsp] | 
** unpckh xmm0, [rsp], xmm2
** lps xmm0, [rsp], xmm2
** vblendvps xmm0, xmm1, xmm2
=> 
** vblendvpd xmm0, xmm1, xmm2
blendvps xmm0, [rsp], xmm0 | 
blendvpd xmm0, [rsp], xmm0 | 
blendps xmm0, [rsp], 0x41 | 
vmaskmovps xmm0, xmm1, [rsp] | 
=> 
sqrtss xmm0, [rsp] | 
sqrtpd xmm0, [rsp] | 
sqrtsd xmm0, [rsp] | 
movhps xmm0, [rsp] | 
** vmovups/d [rsp], xmm1
=> 
vmovaps [rsp], xmm1 | 
cmp [rsp], rcx | 
movsd [rsp], xmm1 | 
movss [rsp], xmm1 | 
movlpd xmm0, [rsp] | 
=> 
movlps xmm0, [rsp] | 
** sub(s,us) xmm0, [rsp], xmm2
** b/w/d/q xmm0, [rsp], xmm2
** padd xmm0, [rsp], xmm2
** cpd2pi xmm0, xmm1
=> 
rcpss xmm0, [rsp] | 
rcpps xmm0, [rsp] | 
** and xmm0, [rsp], xmm2
** andn xmm0, [rsp], xmm2
** xorps xmm0, [rsp], xmm2
=> 
rsqrtss xmm0, [rsp] | 
rsqrtps xmm0, [rsp] | 
blendpd xmm0, [rsp], 0x41 | 
** vcvt(t) xmm0, [rsp]
ret 0x41 | 
=> 
** vcps2dq xmm0, [rsp]
** lbw xmm0, [rsp], xmm2
** pshufw xmm0, [rsp], 0x41
** pshufl xmm0, [rsp], 0x41
** psh

roundss xmm0, [rsp], 0x41 | 
=> 
** roups/d xmm0, [rsp], 0x41
** cvt(t) xmm0, [rsp]
** cpd2pi xmm0, [rsp]
cld  | 
std  | 
=> 
packsswb xmm0, xmm1 | 
** haddps xmm0, xmm1, xmm2
lodsb  | 
lodsw  | 
rcr rbx, 0x1 | 
=> 
rcl rbx, 0x1 | 
** daa 
packsswb xmm0, [rsp] | 
packuswb xmm0, [rsp] | 
vpmaskmovd [rsp], xmm1, xmm2 | 
=> 
** dhsubps xmm0, xmm1, xmm2
** fucomi(p) rbx
** fimul [rsp]
** fidiv(r) [rsp]
fldcw [rsp] | 
=> 
pop rsp | 
fnstcw [rsp] | 
** fadd(p)fsub(r)(p) [rsp]
cvtsd2ss xmm0, [rsp] | 
** vcvtpd2ps xmm0, [rsp]
=> 
cvtpd2ps xmm0, [rsp] | 
dppd xmm0, xmm1, 0x41 | 
** cvt(t)pd2dq xmm0, [rsp]
fnstsw [rsp] | 
movbe rbx, [rsp] | 
=> 
imul bx, [rsp], 0x41 | 
mpsadbw xmm0, [rsp], 0x41 | 
ptest xmm0, [rsp] | 
pcmpistrm xmm0, xmm1, 0x41 | 
aesimc xmm0, [rsp] | 
=> 
mulx ebx, ecx, edx | 
** aad 
pcmpistri xmm0, xmm1, 0x41 | 
** pmulld xmm0, [rsp], xmm2
** vpblendvb xmm0, xmm1, [rsp]
=> 
** stos 
mulx rbx, rcx, [rsp] | 
** fisttp [rsp]
** scas 
** ficom(p) [rsp]
=> 
movbe bx, [rsp] | 
call